In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from numpy.polynomial import chebyshev
from scipy.interpolate import CubicSpline, BarycentricInterpolator, RegularGridInterpolator, PchipInterpolator
from scipy.optimize import minimize_scalar
from scipy.special import roots_legendre
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import Markdown, display

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'figure.dpi': 130, 'font.size': 12, 'axes.titlesize': 'x-large',
    'axes.labelsize': 'large', 'xtick.labelsize': 'medium', 'ytick.labelsize': 'medium'})
np.set_printoptions(suppress=True, linewidth=120)

# --- Utility Functions ---
def note(msg, **kwargs):
    display(Markdown(f"<div class='alert alert-info'>📝 {textwrap.fill(msg, width=100)}</div>"))
def sec(title):
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized.")

# Part 2: Core Numerical Methods
## Chapter 2.6: Interpolation and Function Approximation

### Table of Contents
1.  [Theoretical Foundations](#1.-Theoretical-Foundations)
    *   [1.1 The Weierstrass Approximation Theorem](#1.1-The-Weierstrass-Approximation-Theorem)
    *   [1.2 The Runge Phenomenon: A Cautionary Tale](#1.2-The-Runge-Phenomenon:-A-Cautionary-Tale)
2.  [1D Function Approximation Methods](#2.-1D-Function-Approximation-Methods)
    *   [2.1 Orthogonal Polynomials and Chebyshev Approximation](#2.1-Orthogonal-Polynomials-and-Chebyshev-Approximation)
    *   [2.2 Spline Interpolation and Shape Preservation](#2.2-Spline-Interpolation-and-Shape-Preservation)
3.  [Numerical Integration: Gaussian Quadrature](#3.-Numerical-Integration:-Gaussian-Quadrature)
4.  [Application: Solving for Optimal Policy Functions](#4.-Application:-Solving-for-Optimal-Policy-Functions)
5.  [The Curse of Dimensionality and Sparse Grids](#5.-The-Curse-of-Dimensionality-and-Sparse-Grids)
6.  [Chapter Summary](#6.-Chapter-Summary)
7.  [Exercises](#7.-Exercises)

### Introduction: Solving Models by Filling in the Gaps

In many economic models, particularly in dynamic programming, we cannot solve for the value function or policy function analytically. Instead, solving the Bellman equation via methods like value function iteration gives us the value of the function only at a discrete set of points on a state-space grid. To find the optimal action at a state that falls *between* these grid points, we need a way to accurately "fill in the gaps." This is the core task of **interpolation** and **function approximation**.

This notebook introduces the robust, professional methods that form the backbone of modern computational economics. We will see how naive approaches can fail spectacularly and explore the two workhorse methods for high-quality approximation: **Chebyshev regression** and **cubic spline interpolation**. We will also explore methods for multi-dimensional and unstructured data, grounding our exploration in core economic applications.

### 1. Theoretical Foundations

#### 1.1 The Weierstrass Approximation Theorem
Before attempting to approximate a function, we should ask: is a good approximation even possible? The **Weierstrass Approximation Theorem** provides the fundamental justification. It states that for any continuous function $f(x)$ defined on a closed interval $[a, b]$, there exists a sequence of polynomials that converges uniformly to $f(x)$.

In essence, the theorem guarantees that we can get arbitrarily close to any continuous function with a polynomial of a sufficiently high degree. This gives us the theoretical license to pursue polynomial approximation as a valid strategy. The challenge, as we will see, is how to construct this polynomial in a stable and efficient way.

#### 1.2 The Runge Phenomenon: A Cautionary Tale
A natural first thought for approximating a function is to fit a single, high-degree polynomial that passes through all our known data points. This approach has a fatal flaw, first identified by Carl Runge. He showed that for many simple, well-behaved functions, as you increase the degree of the interpolating polynomial on *evenly-spaced nodes*, the approximation develops wild oscillations near the endpoints of the interval. This is known as the **Runge phenomenon**.

In [ ]:
sec("Demonstrating Runge's Phenomenon")
def runge_func(x): return 1 / (1 + 25 * x**2)
x_dense = np.linspace(-1, 1, 400); y_true = runge_func(x_dense)
nodes_uniform = np.linspace(-1, 1, 11)
poly_high_degree = BarycentricInterpolator(nodes_uniform, runge_func(nodes_uniform))

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(x_dense, poly_high_degree(x_dense), label=f'Degree {len(nodes_uniform)-1} Polynomial Approx.', lw=2)
ax.plot(x_dense, y_true, 'k--', lw=2, label='True Function')
ax.plot(nodes_uniform, runge_func(nodes_uniform), 'ro', ms=8, label='Evenly-Spaced Nodes')
ax.set_title("Runge's Phenomenon"); ax.set_xlabel('x')
ax.set_ylabel('y'); ax.legend(); ax.set_ylim(-0.5, 1.5)
plt.show()

### 2. 1D Function Approximation Methods

#### 2.1 Orthogonal Polynomials and Chebyshev Approximation
The Runge phenomenon is caused by using evenly-spaced nodes. The cure is to use nodes that are clustered near the boundaries of the interval. The optimal choice for this is the **Chebyshev nodes**.

Chebyshev polynomials are part of a broader class of **orthogonal polynomials**. Two polynomials $P_n(x)$ and $P_m(x)$ are orthogonal with respect to a weight function $w(x)$ on an interval $[a, b]$ if their inner product is zero: $\int_a^b P_n(x) P_m(x) w(x) dx = 0$ for $n \ne m$. This orthogonality is analogous to the orthogonality of vectors. When we use the values of orthogonal polynomials as basis functions in a regression, the resulting system of equations is well-conditioned, leading to a stable and accurate fit. This is the foundation of **Chebyshev regression**.

In [ ]:
sec("Approximating with Chebyshev Polynomials")
degree = 11; domain = [-1, 1]
cheb_nodes = chebyshev.chebpts1(degree)
y_at_cheb_nodes = runge_func(cheb_nodes)
coeffs = chebyshev.chebfit(cheb_nodes, y_at_cheb_nodes, deg=degree-1)
cheb_approx = chebyshev.Chebyshev(coeffs, domain=domain)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(x_dense, y_true, 'k--', lw=2, label='True Function')
ax.plot(x_dense, cheb_approx(x_dense), label=f'Degree {degree-1} Chebyshev Approx.')
ax.plot(cheb_nodes, y_at_cheb_nodes, 'o', ms=8, label='Chebyshev Nodes')
ax.set_title("Chebyshev Approximation of Runge's Function"); ax.legend(); ax.set_ylim(-0.5, 1.5)
plt.show()

#### 2.2 Spline Interpolation and Shape Preservation
An alternative to a single high-degree polynomial is to use many low-degree polynomials to connect points. 
- **Cubic Spline:** A series of piecewise cubic polynomials connected such that the entire curve is twice continuously differentiable. This provides excellent smoothness but can overshoot the data.
- **PCHIP (Piecewise Cubic Hermite Interpolating Polynomial):** A special type of spline that is guaranteed to preserve the monotonicity of the original data points. This is essential when approximating objects like CDFs or monotonic value functions.

### 3. Numerical Integration: Gaussian Quadrature
A problem closely related to function approximation is **numerical integration** (or quadrature): approximating the value of a definite integral $\int_a^b f(x) dx$. Simple methods like the Trapezoidal rule can be slow to converge. **Gaussian Quadrature** is a far more powerful and efficient method.

The key insight is that by strategically choosing the locations (nodes) where we evaluate the function, we can achieve a much higher degree of accuracy. Gaussian quadrature chooses the $n$ evaluation nodes to be the roots of the $n$-th degree Legendre polynomial (a class of orthogonal polynomials). With this choice of nodes, an $n$-point Gaussian quadrature rule can exactly integrate any polynomial of degree $2n-1$ or less. This is dramatically more accurate than a simple Riemann sum with the same number of function evaluations.

In [ ]:
sec("Gaussian Quadrature vs. Trapezoidal Rule")
f = lambda x: np.exp(np.sin(x))
a, b = 0, np.pi
true_integral = 3.9774632605
n_points = 5

# 1. Trapezoidal Rule
x_trap = np.linspace(a, b, n_points)
integral_trap = np.trapz(f(x_trap), x_trap)

# 2. Gaussian Quadrature
nodes, weights = roots_legendre(n_points)
# Scale nodes and weights from [-1, 1] to [a, b]
nodes_scaled = 0.5*(b-a)*nodes + 0.5*(b+a)
weights_scaled = 0.5*(b-a)*weights
integral_gauss = np.sum(weights_scaled * f(nodes_scaled))

note(f"For n={n_points} points:")
print(f"Trapezoidal Rule: Integral = {integral_trap:.8f}, Error = {abs(integral_trap - true_integral):.2e}")
print(f"Gaussian Quadrature: Integral = {integral_gauss:.8f}, Error = {abs(integral_gauss - true_integral):.2e}")
note("Gaussian Quadrature is orders of magnitude more accurate for the same number of function evaluations.")

### 4. Application: Solving for Optimal Policy Functions
A core application of function approximation is solving dynamic programming problems. Consider a simple consumption-savings model where an agent lives for `T` periods, has utility $u(c) = \log(c)$, and wants to maximize lifetime utility. The Bellman equation is:
$$ V_t(W_t) = \max_{c_t} \left\{ \log(c_t) + \beta V_{t+1}(W_{t+1}) \right\} $$
We solve this backwards. At each period $t$, we need a continuous approximation of the next period's value function, $\hat{V}_{t+1}$. We use this to solve for the optimal consumption $c_t^*(W)$ for a grid of wealth levels. We then create a continuous approximation of this **policy function**, $\hat{c}_t^*(W)$, to use in the next step of the backward induction.

In [ ]:
sec("Solving for the Optimal Consumption Policy")
T = 10; beta = 0.95; r = 0.05
w_grid = np.linspace(1, 100, 50)
u = lambda c: np.log(c)

V_next = u(w_grid)
policy_funcs = {}

for t in range(T - 1, -1, -1):
    V_approx = PchipInterpolator(w_grid, V_next)
    c_policy = np.empty_like(w_grid)
    V_curr = np.empty_like(w_grid)
    
    for i, w in enumerate(w_grid):
        def objective(c): return -(u(c) + beta * V_approx((w - c) * (1 + r)))
        res = minimize_scalar(objective, bounds=(1e-6, w), method='bounded')
        c_policy[i] = res.x
        V_curr[i] = -res.fun
        
    policy_funcs[t] = PchipInterpolator(w_grid, c_policy)
    V_next = V_curr

note("Plotting the calculated optimal policy functions (consumption rules):")
fig, ax = plt.subplots(figsize=(10, 6))
w_dense = np.linspace(1, 100, 200)
for t in range(0, T, 2):
    ax.plot(w_dense, policy_funcs[t](w_dense), label=f'$c_{t}^*(W)$')
ax.set_title('Optimal Policy Functions Over Time')
ax.set_xlabel('Wealth (W)'); ax.set_ylabel('Optimal Consumption (c*)')
ax.legend(); plt.show()

### 5. The Curse of Dimensionality and Sparse Grids
The number of grid points required for grid-based interpolation grows exponentially with the number of dimensions. This is the **curse of dimensionality**. A 10-point grid in 1D requires 10 function evaluations. In 5D, a full tensor grid requires $10^5 = 100,000$ points. This is computationally infeasible.

**Sparse grids** provide a powerful method for mitigating this curse. A Smolyak sparse grid is constructed by taking a clever linear combination of tensor-product grids at different levels of resolution. The result is a grid that has far fewer points than a full tensor grid but still maintains good accuracy, especially for smooth functions where interactions between dimensions are not too complex.

In [ ]:
sec("Visualizing a Smolyak Sparse Grid vs. a Tensor Grid")
from itertools import product

# Simple implementation for visualization purposes
def cheb_nodes_1d(n): return np.cos(np.pi * (2*np.arange(1, n+1) - 1) / (2*n))

level = 4
tensor_nodes = cheb_nodes_1d(2**level + 1)
tx, ty = np.meshgrid(tensor_nodes, tensor_nodes)

smolyak_x, smolyak_y = [], []
for i in range(1, level + 2):
    nodes_i = cheb_nodes_1d(2**(i-1)+1 if i>1 else 1)
    for j in range(1, level + 2 - i):
        nodes_j = cheb_nodes_1d(2**(j-1)+1 if j>1 else 1)
        for p in product(nodes_i, nodes_j):
            smolyak_x.append(p[0]); smolyak_y.append(p[1])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
ax1.plot(tx, ty, 'bo', ms=4); ax1.set_title(f'Tensor Grid ({(2**level+1)**2} points)')
ax2.plot(smolyak_x, smolyak_y, 'ro', ms=4); ax2.set_title(f'Smolyak Grid ({len(smolyak_x)} points)')
for ax in [ax1, ax2]: ax.set_aspect('equal')
plt.show()

### 6. Chapter Summary
- **Theoretical Backing:** The **Weierstrass Theorem** guarantees that polynomial approximation is possible. However, naive interpolation on evenly-spaced nodes fails due to the **Runge phenomenon**.
- **1D Workhorses:** For high-accuracy global approximation, **Chebyshev regression** (using orthogonal polynomials on Chebyshev nodes) is the gold standard. For local, smooth, and shape-preserving interpolation, **cubic splines** and **PCHIP** are preferred.
- **Numerical Integration:** **Gaussian Quadrature** provides a highly efficient and accurate method for numerical integration by strategically placing evaluation nodes at the roots of orthogonal polynomials.
- **Dynamic Programming:** A primary application of these methods is solving for **value functions** and **policy functions** in dynamic models via backward induction.
- **Curse of Dimensionality:** For problems with more than a few state variables, full grid-based methods become infeasible. **Sparse grids** offer a powerful way to mitigate this curse, and even more advanced methods exist for higher dimensions.

### 7. Exercises

1.  **Chebyshev Implementation:** Write a function `chebyshev_nodes(n, a, b)` that computes the `n` Chebyshev nodes on an arbitrary interval `[a, b]`. Use it to find the nodes for approximating a function on `[-5, 5]` with 20 nodes.

2.  **Impact of Curvature:** In the dynamic savings problem, the utility function's curvature is governed by risk aversion. Re-solve the model using a CRRA utility function $u(c) = \frac{c^{1-\gamma}-1}{1-\gamma}$ for `gamma=2` and `gamma=5`. Plot the resulting *policy functions*. How does higher risk aversion (more curvature) affect the optimal consumption policy? Provide an economic intuition.

3.  **Gaussian Quadrature Error:** Write a script to compute the integral of $f(x) = e^x$ from 0 to 1 using Gaussian Quadrature. Plot the absolute error of the approximation as a function of the number of quadrature points `n` (from 1 to 10). What do you observe about the rate of convergence?

4.  **Extrapolation Dangers:** Using the `cheb_approx` and `spline_approx` objects from the notebook, evaluate and plot their approximations on a wider domain, from `x = -1.5` to `x = 1.5`. How do the approximations behave outside the original interpolation range `[-1, 1]`? Which method appears more dangerously wrong in extrapolation?

5.  **2D Interpolation:** A firm's profit function is given by $\pi(p, w) = (100 - 2p)p - 5w(100-2p)^{0.5}$, where `p` is price and `w` is the wage. You have evaluated this function on a coarse 10x10 grid for `p` in `[10, 40]` and `w` in `[5, 20]`. Use `RegularGridInterpolator` to create a smooth approximation of the profit surface and find the price and wage that maximize profits on your interpolated surface.